# Preprocessing Pipeline

In [1]:
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np
import os

In [2]:
def date_fixer(x):
    return x[:2]+'-'+x[2:4]+'-'+x[4:8]

def pipeline(url):
    #to aid in imputing the values that do not change
    map_dict = {'100':[4,700000,200000,1000,1203],'200':[6,750000,250000,1250,2400],
     '750':[4,1500000,500000,9300,1700],'1500':[4,3000000,1000000,18500,1700],
     '7500':[4,15000000,5000000,93000,1700]}
    
    #TO READ THE FILE
    f = urlopen(url) #to link towards the URL
    my_file = f.read() #to read the lines
    s=str(my_file) #convert into string
    ss=s.split('\\n') #for parsing

    #TO FORMAT THE DETAILS TO BE EXTRACTED
    det_ph = [] #place holder list for the details reagrding the draw
    for i in ss[:32]: #since we only need details
        ## fresh list
        new_lst=[]
        for n in i:
            try:
                #to make sure only the numerical values pass by
                ii = int(n)
                new_lst.append(n) #append the filtered numerical values
            except:
                pass
        if len(new_lst)>0: #to delete empty lists
            det_ph.append(''.join(new_lst)) #to join all entries of a single line
    
    #TO EXTRACT USEFUL DETAILS
    den = det_ph[0] #denomination
    draw_num = det_ph[1] #draw number
    date = det_ph[2] #date of the draw
    fst_pr = map_dict[den][1] #first price
    sec_pr = map_dict[den][2] #second price
    trd_pr = map_dict[den][3] #third price

    #TO EXTRACT WINNING NUMBERS
    jnd = [] #another fresh list
    for i in ss:
        sp = i.split('\\t') #to extract the numbers
        new_lst=[]
        for n in sp:
            split = np.array(n.replace('\\r',' ').split())
            for n2 in split: 
                try:
                    check = n2.astype(np.uint32)
                    if len(n2) >5:
                        new_lst.append(n2)
                except:
                    pass
        if len(new_lst)>0:
            jnd.append(new_lst)
    num=[]
    for f in jnd:
        for s in f:
            if len(s)>5:
                num.append(s)  
    
    #TO COMPILE THE DATA INTO FRAME
    df=pd.DataFrame(num,columns=['Bond No.']) #to create an empty frame
    ind = map_dict[den][0] #this will map the indices to extract second prize winners
    df['Price'] = '' #create an empty price column
    df.iloc[0,1] = 'first' #fill the values
    df.iloc[1:ind,1] = 'second'
    df.iloc[ind:,1] = 'third'
    df['Amount'] = '' #create empty amount column
    df.iloc[0,2] = fst_pr #fill the values
    df.iloc[1:ind,2] = sec_pr
    df.iloc[ind:,2] = trd_pr
    df['Denomination'] = den
    df['Draw No.'] = draw_num
    df['Draw date'] = date
    df['Draw date']=df['Draw date'].apply(lambda x : date_fixer(x))
    df = df[df['Bond No.'] != '000000']
    
    #TO FIX MORE FORMATTING RELATED ISSUES
    if not len(df) == map_dict[den][4]:
        mdf = pd.DataFrame().reindex_like(df).iloc[:ind,:]
        mdf.iloc[0,0] = det_ph[4]
        mdf.iloc[1:,0] = [det_ph[6][6*(i):6*(i+1)] for i in range(len(det_ph[6])//6)]
        mdf.iloc[0,1] = 'first'
        mdf.iloc[1:,1] = 'second'
        mdf.iloc[0,2] = fst_pr
        mdf.iloc[1:,2] = sec_pr
        mdf.iloc[:,3] = den
        mdf.iloc[:,4] = draw_num
        mdf.iloc[:,5] = date
        df.iloc[:,1] = 'third'
        df = pd.concat([mdf,df])
    return df

In [3]:
with open('link_lst.txt') as f: #to go through all the links one by one
    r = f.read().split('\n')

In [4]:
dff = pd.DataFrame(columns=['Bond No.', 'Price', 'Amount', 'Denomination', 'Draw No.', 'Draw date'])
#to make one dataframe with complete data
for c,u in enumerate(r):
    d = pipeline(u)
    print('Joined',c+1,'Frame','the last denomination was',d.iloc[0,3])
    dff = pd.concat([dff,d])

Joined 1 Frame the last denomination was 100
Joined 2 Frame the last denomination was 100
Joined 3 Frame the last denomination was 100
Joined 4 Frame the last denomination was 100
Joined 5 Frame the last denomination was 100
Joined 6 Frame the last denomination was 100
Joined 7 Frame the last denomination was 100
Joined 8 Frame the last denomination was 100
Joined 9 Frame the last denomination was 100
Joined 10 Frame the last denomination was 100
Joined 11 Frame the last denomination was 100
Joined 12 Frame the last denomination was 100
Joined 13 Frame the last denomination was 100
Joined 14 Frame the last denomination was 100
Joined 15 Frame the last denomination was 100
Joined 16 Frame the last denomination was 100
Joined 17 Frame the last denomination was 100
Joined 18 Frame the last denomination was 100
Joined 19 Frame the last denomination was 100
Joined 20 Frame the last denomination was 100
Joined 21 Frame the last denomination was 100
Joined 22 Frame the last denomination was 1

In [12]:
dff.to_csv('data.csv')
## with this, the process of data collection is completed